In [ ]:
import json
from requests import request
from requests.compat import urljoin, urlparse
from requests.exceptions import HTTPError
from urllib.robotparser import RobotFileParser
from bs4 import BeautifulSoup
from time import sleep
import re
from requests.compat import urlparse, urljoin
import sqlite3
from requests import Session
import datetime

In [ ]:
def canfetch(url, agent='*', path='/'):
    robot = RobotFileParser(urljoin(url, '/robots.txt'))
    robot.read()
    return robot.can_fetch(agent, urlparse(url)[2])
    
def download(url, params={}, headers={}, method='GET', limit=3):
    try:
        session = Session()
        resp = session.request(method, url,
                               params = params if method.upper() == 'GET' else '',
                               data = params if method.upper() == 'POST' else '',
                               headers = headers)
        resp.raise_for_status()
    except HTTPError as e:
        if limit > 0 and e.response.status_code >= 500:
            print(limit)
            time.sleep(60) # Server Error이기 때문에 delay를 두고 실행하기 위해서 사용한다.
            # 보통은 5분에 한 번꼴로 random하게 되도록 설정한다.
            download(url, params, headers, method, limit-1)
        else:
            print('[{}] '.format(e.response.status_code) + url)
            print(e.response.reason)
            print(e.response.headers)
    return resp

In [ ]:
url_inbang = 'https://gall.dcinside.com/board/lists/?id=ib_new1&page=1'
url_m = 'https://gall.dcinside.com/board/lists/?id=m_entertainer1'
url_w = 'https://gall.dcinside.com/board/lists?id=w_entertainer'
url_instiz = 

In [ ]:
# conn = sqlite3.connect('Entertainment.db')
# cur = conn.cursor()
# cur.executescript('''
#     DROP TABLE IF EXISTS head;
#     CREATE TABLE head(
#         pk INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
#         head TEXT NOT NULL,
#         cdate TEXT NOT NULL,
#         wdate TEXT NOT NULL,
#         ref INTEGER NOT NULL,
#         page INTEGER NOT NULL
#     );
# ''')

# cur.executescript('''
#     DROP TABLE IF EXISTS history;
#     CREATE TABLE history(
#         pk INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
#         seen TEXT NOT NULL,
#         ref INTEGER NOT NULL
#     );
# ''')

In [ ]:
# 인방갤
# 처음 시작할 때 
url_inbang = 'https://gall.dcinside.com/board/lists/?id=ib_new1&page=1'
seen = list()

headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'}
urls = list()


urls.append(url_inbang)

count = 0


while urls:
    try:
        count += 1
        seed = urls.pop(0)

        resp = download(seed, headers=headers)
        dom = BeautifulSoup(resp.text, 'html.parser')
        seen.append(seed)
        cur.execute('''
            INSERT INTO history(seen, ref) VALUES(?, 100)
        ''', [seed])

        for _ in [_['href'] for _ in dom.select('div.bottom_paging_box > a')[:-1]  #끝 미포함
                  if _.has_attr('href')and re.match(r'(?:https?:/)?/\w+(?:[./]\w+)+',_['href']).group()
                 and _.text.strip() != '처음' and  _.text.strip() != '이전']:
            newUrls = urljoin(seed, _)
            if newUrls not in urls and newUrls not in seen:
                 urls.append(newUrls)

        if dom.select('tr.ub-content.us-post > td:nth-of-type(2) > a:nth-of-type(1)') != None:
            head = [_.text.strip() for _ in dom.select('tr.ub-content.us-post > td:nth-of-type(2) > a:nth-of-type(1)') ]
            wdate = [_['title'] for _ in dom.select('tr.ub-content.us-post > td.gall_date')]
            cdate = str(datetime.datetime.now()).split('.')[0]
            page = re.search('page=(\d+)', urlparse(seed).query).group(1)
            if len(head)==len(wdate):
                for _ in range(0,len(head)):
                    cur.execute('''
                        INSERT INTO head(head, cdate, wdate, page, ref) VALUES(?,?,?,?,100)
                        ''', [head[_], wdate[_], cdate, page])
                    conn.commit()

        if count%100 == 0:
                print(count)
     
    except Exception as e:
            print('Error',e)

In [ ]:
conn = sqlite3.connect('Entertainment.db')
cur = conn.cursor()
# 인방갤
# 이어서 시작할 때
cur.execute('SELECT seen FROM history')
seen = cur.fetchall()
seen_ = list()
for _ in seen:
    seen_.append(''.join(_))
url_inbang = seen[-1]
url_inbang = ''.join(url_inbang)

In [ ]:
headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'}
urls = list()


urls.append(url_inbang)

count = 0


while urls:
    try:
        count += 1
        seed = urls.pop(0)

        resp = download(seed, headers=headers)
        dom = BeautifulSoup(resp.text, 'html.parser')
        seen_.append(seed)
        cur.execute('''
            INSERT INTO history(seen, ref) VALUES(?, 100)
        ''', [seed])

        for _ in [_['href'] for _ in dom.select('div.bottom_paging_box > a')[:-1]  #끝 미포함
                  if _.has_attr('href')and re.match(r'(?:https?:/)?/\w+(?:[./]\w+)+',_['href']).group()
                 and _.text.strip() != '처음' and  _.text.strip() != '이전']:
            newUrls = urljoin(seed, _)
            if newUrls not in urls and newUrls not in seen_:
                 urls.append(newUrls)
        
        
        if dom.select('tr.ub-content.us-post > td:nth-of-type(2) > a:nth-of-type(1)') != None:
            head = [_.text.strip() for _ in dom.select('tr.ub-content.us-post > td:nth-of-type(2) > a:nth-of-type(1)') ]
            wdate = [_['title'] for _ in dom.select('tr.ub-content.us-post > td.gall_date')]
            cdate = str(datetime.datetime.now()).split('.')[0]
            page = re.search('page=(\d+)', urlparse(seed).query).group(1)
            if len(head)==len(wdate):
                for _ in range(0,len(head)):
                    cur.execute('''
                        INSERT INTO head(head, cdate, wdate, page, ref) VALUES(?,?,?,?,100)
                        ''', [head[_], wdate[_], cdate, page])
                    conn.commit()

        if count%100 == 0:
                print(count)
     
    except Exception as e:
            print('Error',e)

In [ ]:
#남자 연예인 갤러리
url_man = 'https://gall.dcinside.com/board/lists/?id=m_entertainer1&page=1'
seen = list()

headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'}
urls = list()


urls.append(url_man)

count = 0


while urls:
    try:
        count += 1
        seed = urls.pop(0)

        resp = download(seed, headers=headers)
        dom = BeautifulSoup(resp.text, 'html.parser')
        seen.append(seed)
        cur.execute('''
            INSERT INTO history(seen, ref) VALUES(?, 101)
        ''', [seed])

        for _ in [_['href'] for _ in dom.select('div.bottom_paging_box > a')[:-1]  #끝 미포함
                  if _.has_attr('href')and re.match(r'(?:https?:/)?/\w+(?:[./]\w+)+',_['href']).group()
                 and _.text.strip() != '처음' and  _.text.strip() != '이전']:
            newUrls = urljoin(seed, _)
            if newUrls not in urls and newUrls not in seen:
                 urls.append(newUrls)

        if dom.select('tr.ub-content.us-post > td:nth-of-type(2) > a:nth-of-type(1)') != None:
            head = [_.text.strip() for _ in dom.select('tr.ub-content.us-post > td:nth-of-type(2) > a:nth-of-type(1)') ]
            wdate = [_['title'] for _ in dom.select('tr.ub-content.us-post > td.gall_date')]
            cdate = str(datetime.datetime.now()).split('.')[0]
            page = re.search('page=(\d+)', urlparse(seed).query).group(1)
            if len(head)==len(wdate):
                for _ in range(0,len(head)):
                    cur.execute('''
                        INSERT INTO head(head, cdate, wdate, page, ref) VALUES(?,?,?,?,101)
                        ''', [head[_], wdate[_], cdate, page])
                    conn.commit()

        if count%100 == 0:
                print(count)
     
    except Exception as e:
            print('Error',e)

In [ ]:
#여자 연예인 갤러리
url_woman = 'https://gall.dcinside.com/board/lists?id=w_entertainer&page=1'
seen = list()

headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'}
urls = list()


urls.append(url_woman)

count = 0


while urls:
    try:
        count += 1
        seed = urls.pop(0)

        resp = download(seed, headers=headers)
        dom = BeautifulSoup(resp.text, 'html.parser')
        seen.append(seed)
        cur.execute('''
            INSERT INTO history(seen, ref) VALUES(?, 102)
        ''', [seed])

        for _ in [_['href'] for _ in dom.select('div.bottom_paging_box > a')[:-1]  #끝 미포함
                  if _.has_attr('href')and re.match(r'(?:https?:/)?/\w+(?:[./]\w+)+',_['href']).group()
                 and _.text.strip() != '처음' and  _.text.strip() != '이전']:
            newUrls = urljoin(seed, _)
            if newUrls not in urls and newUrls not in seen:
                 urls.append(newUrls)

        if dom.select('tr.ub-content.us-post > td:nth-of-type(2) > a:nth-of-type(1)') != None:
            head = [_.text.strip() for _ in dom.select('tr.ub-content.us-post > td:nth-of-type(2) > a:nth-of-type(1)') ]
            wdate = [_['title'] for _ in dom.select('tr.ub-content.us-post > td.gall_date')]
            cdate = str(datetime.datetime.now()).split('.')[0]
            page = re.search('page=(\d+)', urlparse(seed).query).group(1)
            if len(head)==len(wdate):
                for _ in range(0,len(head)):
                    cur.execute('''
                        INSERT INTO head(head, cdate, wdate, page, ref) VALUES(?,?,?,?,102)
                        ''', [head[_], wdate[_], cdate, page])
                    conn.commit()

        if count%100 == 0:
                print(count)
     
    except Exception as e:
            print('Error',e)

In [ ]:
conn = sqlite3.connect('Entertainment.db')
cur = conn.cursor()

In [ ]:
# 인스티즈

url = 'https://www.instiz.net/name_enter?page=1&category=2'

urls = list()
seen = list()
urls.append(url)

count = 0

while urls:
    
    try:
        count += 1
        seed = urls.pop(0)

        resp = download(url)
        dom = BeautifulSoup(resp.text, 'html.parser')
        seen.append(seed)
        cur.execute('''
            INSERT INTO history(seen, ref) VALUES(?, 103)
        ''', [seed])
        

        for _ in [_['href'] for _ in dom.select('#indextable a')
                  if _.has_attr('href') and re.match(r'(../)?\w+', _['href']).group()]:
            newUrls = urljoin(seed, _)
            if newUrls not in urls and newUrls not in seen:
                urls.append(newUrls)
                
        if dom.select('#subject > a') != None:
            head = [_.text.strip() for _ in dom.select('#subject > a')]
            wdate = [_.text.strip() for _ in dom.select('td.listno.regdate')]
            cdate = str(datetime.datetime.now()).split('.')[0]
            page = re.search('p?a?g?e?=?(\d+)', urlparse(seed).query).group(1)

            for _ in range(0,len(head)):
                cur.execute('''
                    INSERT INTO head(head, cdate, wdate, page, ref) VALUES(?,?,?,?,103)
                    ''', [head[_], wdate[_], cdate, page])
                conn.commit()

        if count % 100 == 0:
            print(count)
    
    except Exception as e:
            print('Error',e)